In [1]:
import os
from dotenv import load_dotenv
from typing import TypedDict, Union
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI

# Load API keys and set up tracing
load_dotenv()
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGSMITH_PROJECT"] = "Intro to LangGraph"

# Define the different data structures (schemas) for our tasks
class MathState(TypedDict):
    question: str
    answer: int

class GeneralState(TypedDict):
    question: str
    answer: str

# The overall state can be a union of the different schemas
class GraphState(TypedDict):
    route: str
    data: Union[MathState, GeneralState]

# --- Define the nodes for our graph ---
def solve_math(state: GraphState):
    print("--- Solving Math Problem ---")
    question = state['data']['question']
    
    # A simple way to solve math problems for this example
    try:
        answer = eval(question)
    except:
        answer = "I can't solve that math problem."
        
    return {"data": {"answer": answer}}

def answer_general(state: GraphState):
    print("--- Answering General Question ---")
    question = state['data']['question']
    
    llm = ChatOpenAI(model="gpt-4o")
    response = llm.invoke(f"Answer the following question: {question}")
    
    return {"data": {"answer": response.content}}

# --- Define the router ---
# This function decides which node to go to next
def route_question(state: GraphState):
    question = state['data']['question']
    if any(op in question for op in ['+', '-', '*', '/']):
        # If it's a math question, go to the math node
        return "solve_math"
    else:
        # Otherwise, go to the general QA node
        return "answer_general"

# --- Build the graph ---
workflow = StateGraph(GraphState)

# Add the nodes
workflow.add_node("solve_math", solve_math)
workflow.add_node("answer_general", answer_general)

# The entry point is a conditional edge that calls our router
workflow.add_conditional_edges(START, route_question, {
    "solve_math": "solve_math",
    "answer_general": "answer_general"
})

# Both nodes lead to the end of the graph
workflow.add_edge("solve_math", END)
workflow.add_edge("answer_general", END)

# Compile the graph into a runnable app
app = workflow.compile()

# --- Run the graph with different inputs to test the router ---
print("--- Testing Math Route ---")
math_input = {"data": {"question": "5 * 8"}}
result = app.invoke(math_input)
print(f"Answer: {result['data']['answer']}")

print("\n--- Testing General Route ---")
general_input = {"data": {"question": "What is LangGraph?"}}
result = app.invoke(general_input)
print(f"Answer: {result['data']['answer']}")

--- Testing Math Route ---
--- Solving Math Problem ---
Answer: 40

--- Testing General Route ---
--- Answering General Question ---
Answer: LangGraph is a project developed by OpenAI, designed to map the various connections and relationships between different languages and dialects. The goal of LangGraph is to provide insights into how languages are related to one another, how they have evolved over time, and how they influence each other. This tool can be useful for linguists, researchers, and language enthusiasts interested in understanding the complexities of global linguistic landscapes.

LangGraph typically involves the use of algorithms and data analysis to process large amounts of linguistic data, covering aspects such as grammar, vocabulary, phonetics, and historical language shifts. By exploring these connections, LangGraph can help users visualize the intricate web of linguistic relationships and better appreciate the diversity and interconnectedness of human languages.
